In [1]:
import numpy as np
import pandas as pd

Нашел баг при передаче классам нескольких категориальных переменных и пока не успел исправить, поэтому пока не работает полноценно

In [119]:
class missing_filler_category():
    
    """
    Returns a table with all the missing values filled with special category.
    params::x DataFrame with the data
    params::filling_category a value to fill the missing values
    """
    
    def __init__(self, x, filling_category = "filler", y = None):
        self.x = x
        self.y = y
        self.filling_category = filling_category
        
    def fit(self):
        return self
    
    def transform(self):
        self.x.fillna(self.filling_category, inplace = True)
        filled_table = self.x
        return filled_table
    
    
class missing_filler_mean():
    
    """
    Returns a table with all the missing values in numerical columns filled with mean and all the 
    missing values in categorical columns filled with special category. If you do not wish to fill
    categorical at all, just do not pass "categorical_variables" parameter.
    params::x DataFrame with the data
    params::categorical_variables list of categorical varaibles
    params::filling_category a value to fill the missing values in categorical variables
    """
    
    def __init__(self, x, categorical_variables = [], filling_category = "filler", y = None):
        self.x = x
        self.y = y
        self.categorical_variables = categorical_variables
        self.filling_category = filling_category
        
    def fit(self):
        self.mean_to_fill = self.x.mean()
        return self
    
    def transform(self):
        filled_table = self.x.fillna(self.mean_to_fill)
        if len(self.categorical_variables) != 0:
            filled_table[self.categorical_variables].fillna(self.filling_category, inplace = True)
        return filled_table
    
    
    
class missing_filler_median():
    
    """
    Returns a table with all the missing values in numerical columns filled with median and all the 
    missing values in categorical columns filled with special category. If you do not wish to fill
    categorical at all, just do not pass "categorical_variables" parameter.
    params::x DataFrame with the data
    params::categorical_variables list of categorical varaibles
    params::filling_category a value to fill the missing values in categorical variables
    """
    
    def __init__(self, x, categorical_variables = [], filling_category = "filler", y = None):
        self.x = x
        self.y = y
        self.categorical_variables = categorical_variables
        self.filling_category = filling_category
        
    def fit(self):
        self.median_to_fill = self.x.median()
        return self
    
    def transform(self):
        filled_table = self.x.fillna(self.median_to_fill)
        if len(self.categorical_variables) != 0:
            print(filled_table[self.categorical_variables])
            filled_table[self.categorical_variables].fillna(self.filling_category, inplace = True)
        return filled_table
    
    
class missing_filler_mode():
    
    """
    Returns a table with all the missing values in numerical columns filled with mode and all the 
    missing values in categorical columns filled with special category. If you do not wish to fill
    categorical at all, just do not pass "categorical_variables" parameter.
    params::x DataFrame with the data
    params::categorical_variables list of categorical varaibles
    params::filling_category a value to fill the missing values in categorical variables
    """
    
    def __init__(self, x, categorical_variables = [], filling_category = "filler", y = None):
        self.x = x
        self.y = y
        self.categorical_variables = categorical_variables
        self.non_categorical_variables = set(list(categorical_variables)).symmetric_difference(list(x.columns))
        self.filling_category = filling_category
        
    def fit(self):
        return self
    
    def transform(self):
        table_to_fill = self.x.copy()
        categorical_table = table_to_fill[self.categorical_variables].copy()
        non_categorical_table = table_to_fill[self.non_categorical_variables].copy()
        
        if len(self.categorical_variables) == 0:
            categorical_table = pd.DataFrame(index = non_categorical_table.index())
        
        categorical_table.fillna(categorical_table.mode().loc[0], inplace = True)
        non_categorical_table.fillna(non_categorical_table.mode().mean(), inplace = True)
        
        filled_table = categorical_table.join(non_categorical_table)
        return filled_table

Проверим, как работает

In [78]:
test_df = pd.DataFrame({"SomeVariable" : [1,2,4,5,None], 
                       "OneMoreVariable" : [12, 66, 12, None, 99], 
                       "SomeCategoricalVariable" : ["Biba", "I", "Boba", "Dva", None]})
test_df

,SomeVariable,OneMoreVariable,SomeCategoricalVariable
0,1.0,12.0,Biba
1,2.0,66.0,I
2,4.0,12.0,Boba
3,5.0,NaN,Dva
4,NaN,99.0,None


In [92]:
test_df[["OneMoreVariable", "SomeCategoricalVariable"]].fillna(test_df.mode().loc[0])

,OneMoreVariable,SomeCategoricalVariable
0,12.0,Biba
1,66.0,I
2,12.0,Boba
3,12.0,Dva
4,99.0,Biba


In [88]:
test_df.mode()[["OneMoreVariable", "SomeCategoricalVariable"]].loc[0]

OneMoreVariable            12.0
SomeCategoricalVariable    Biba
Name: 0, dtype: object

In [77]:
obj = missing_filler_category(test_df, "Grazhdanina")
obj.fit()
obj.transform()

,SomeVariable,OneMoreVariable,SomeCategoricalVariable
0,1.0,12.0,Biba
1,2.0,66.0,I
2,4.0,12.0,Boba
3,5.0,Grazhdanina,Dva
4,Grazhdanina,99.0,Grazhdanina


In [75]:
obj = missing_filler_mean(test_df, "SomeCategoricalVariable", "Grazhdanina")
obj.fit()
obj.transform()

,SomeVariable,OneMoreVariable,SomeCategoricalVariable
0,1.0,12.00,Biba
1,2.0,66.00,I
2,4.0,12.00,Boba
3,5.0,47.25,Dva
4,3.0,99.00,Grazhdanina


In [120]:
obj = missing_filler_median(test_df, ["SomeCategoricalVariable", "OneMoreVariable"], "Grazhdanina")
obj.fit()
obj.transform()

  SomeCategoricalVariable  OneMoreVariable
0                    Biba             12.0
1                       I             66.0
2                    Boba             12.0
3                     Dva             39.0
4                    None             99.0


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,SomeVariable,OneMoreVariable,SomeCategoricalVariable
0,1.0,12.0,Biba
1,2.0,66.0,I
2,4.0,12.0,Boba
3,5.0,39.0,Dva
4,3.0,99.0,None


In [116]:
obj = missing_filler_mode(test_df, "SomeCategoricalVariable", "Grazhdanina")
obj.fit()
obj.transform()

KeyError: "['b', 'C', 'S', 'o', 'c', 'l', 'V', 'a', 'g', 'i', 'm', 'e', 't', 'r'] not in index"